In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(12,8))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import satorchipy as stpy
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
import pickle
from importlib import reload

In [ ]:
reload(dl)
##### Simulation du signal: une gaussienne

# Modulation Source Characteristics
# #### Source characteristics
rf_freq = 150    # GHz
amplitude = 2.   # Volts
offset = 1.5     # Volts
phase = 1.       # radians
frequency = 2  # Hz [Modulation Frequency] 

# Time samples
tmin = 0.
tmax = 100.
samplefreq = 70.
ttin = np.linspace(tmin, tmax, int((tmax-tmin)*samplefreq))

# Input True Signal
t0 = 50.
dt = 10.
yyin = exp(-0.5*(ttin-t0)**2/dt**2)

# Source Signal
signal_src = ll.sim_generator_power(ttin, amplitude, offset, frequency, phase, rf_freq = rf_freq)
signal_src = dl.renorm(signal_src)
#signal_src = -np.sin(ttin*2*np.pi*frequency)


# Measured Signal
noise_level = 0.1
measured = yyin*signal_src + np.random.randn(len(ttin))*noise_level

subplot(2,1,1)
plot(ttin, yyin, label = 'True Signal')
plot(ttin, signal_src-1, label = 'Source Signal', alpha=0.3)
plot(ttin, measured, label = 'Source Signal', alpha=0.5)
ylabel('Input Signal')
xlabel('Time [Sec]')
legend()

subplot(2,1,2)
plot(ttin, yyin, label = 'True Signal')
plot(ttin, signal_src-1, label = 'Source Signal', alpha=0.3)
plot(ttin, measured, label = 'Source Signal', alpha=0.5)
ylabel('Input Signal')
xlabel('Time [Sec]')
xlim(45,55)
legend()

In [ ]:
reload(dl)
### Demodulation Methods
period = 1./frequency

# RMS
newt_rms, amp_rms, errors_rms = dl.demodulate_methods([ttin, measured], 1./period, method='rms', remove_noise=True)

# Fit Sin
newt_sin, amp_sin, errors_sin = dl.demodulate_methods([ttin, measured], 1./period, method='fit')

# Fit True Shape
xxtemplate = np.linspace(0, period, 20)
model_src = ll.sim_generator_power(xxtemplate, amplitude, offset, frequency, phase, rf_freq = rf_freq)
newt_fit, amp_fit, errors_fit = dl.demodulate_methods([ttin, measured], 1./period, method='fit', 
                                                    template=[xxtemplate, model_src])

# Demodulation
newt_demod, amp_demod, errors_demod = dl.demodulate_methods([ttin, measured], 1./period, method='demod',
                                                           src_data_in = [ttin, signal_src])

# Demodulation quadrature
newt_demod_quad, amp_demod_quad, errors_demod_quad = dl.demodulate_methods([ttin, measured], 1./period, 
                                                                           method='demod_quad', remove_noise=True,
                                                                           src_data_in = [ttin, signal_src])


#stop
subplot(2,1,1)
plot(ttin, yyin, 'k', label = 'True Signal')
plot(newt_rms, amp_rms[0,:], label='RMS')
plot(newt_sin, amp_sin[0,:], label='Fit Sine')
plot(newt_fit, amp_fit[0,:], label='Fit True')
plot(newt_demod, amp_demod, label='Demodulation')
plot(newt_demod_quad, amp_demod_quad, label='Demodulation Quadrature')
legend(loc='upper right', fontsize=8)

subplot(2,1,2)
plot(ttin, ttin*0, 'k:', label = 'True Signal')
plot(newt_rms, amp_rms[0,:]-np.interp(newt_rms, ttin, yyin), label='RMS')
plot(newt_sin, amp_sin[0,:]-np.interp(newt_sin, ttin, yyin), label='Fit Sine')
plot(newt_fit, amp_fit[0,:]-np.interp(newt_fit, ttin, yyin), label='Fit True')
plot(newt_demod, amp_demod-np.interp(newt_demod, ttin, yyin), label='Demodulation')
plot(newt_demod_quad, amp_demod_quad-np.interp(newt_demod_quad, ttin, yyin), label='Demodulation Quadrature')
legend(loc='upper right', fontsize=8)
#plot(newt_demod_quad, newt_demod_quad*0+noise_level/sqrt(2))
ylim(-0.5, 0.5)